<a href="https://colab.research.google.com/github/Gaurav0528/summer_school/blob/main/1_GridSearchCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2 align='center' style='color:purple'>Finding best model and hyper parameter tunning using GridSearchCV and RandomisedSearchCV</h2>

**For iris flower dataset in sklearn library, we are going to find out best model and best hyper parameters using GridSearchCV**

**Step1: Load iris flower dataset**

In [ ]:
from sklearn import svm, datasets
import numpy as np
# Loading Iris dataset
iris = datasets.load_iris()

**Step2: Converting data into Dataframe**

In [ ]:
import pandas as pd
# adding feature names to coloumn
df = pd.DataFrame(iris.data,columns=iris.feature_names)
# adding target coloumn name
df['flower'] = iris.target
# mapping class to each flower name
df['flower'] = df['flower'].apply(lambda x: iris.target_names[x])
df[47:150]

<h3 style='color:blue'>Approach 1: Use train_test_split and manually tune parameters by trial and error</h3>

**Step3: Train test split**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
# (test_size=0.3) --> 30% testing data and 70% is training data
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.3)

**Step4: Training the SVM model**

In [ ]:
# creating an instance of SVM classifier
model = svm.SVC(kernel='rbf',C=30,gamma='auto')
# training the model
model.fit(X_train,y_train)
# testing the model
model.score(X_test, y_test)

<h3 style='color:blue'>Approach 2: Use K Fold Cross validation</h3>

**Step5.1: Manually try supplying models and parameters to cross_val_score function with 5 fold cross validation**

In [ ]:
from numpy import mean
# k fold cross validation with cv=5
scores=cross_val_score(svm.SVC(kernel='linear',C=10,gamma='auto'),iris.data, iris.target, cv=5)
# printing scores of each fold
print(scores)
# average score
print(mean(scores))

**Step5.2: trying with different parameters**

In [ ]:
scores=cross_val_score(svm.SVC(kernel='rbf',C=10,gamma='auto'),iris.data, iris.target, cv=5)
print(scores)
print(mean(scores))

**Step5.3: Again trying with different parameters**

In [ ]:
scores=cross_val_score(svm.SVC(kernel='rbf',C=20,gamma='auto'),iris.data, iris.target, cv=5)
print(scores)
print(mean(scores))

**Step6: Above approach is tiresome and very manual. We can use for loop as an alternative**

In [ ]:
# trying different values for kernel value
kernels = ['rbf', 'linear']
C = [1,10,20]
avg_scores = {}
# nested loops for iterating on different values of kernel and C
for kval in kernels:
    for cval in C:
        cv_scores = cross_val_score(svm.SVC(kernel=kval,C=cval,gamma='auto'),iris.data, iris.target, cv=5)
        avg_scores[kval + '_' + str(cval)] = np.average(cv_scores)

avg_scores

<h3 style='color:blue'>Approach 3: Use GridSearchCV</h3>

**Step7: GridSearchCV does exactly same thing as for loop above but in a single line of code**

In [ ]:
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(svm.SVC(gamma='auto'), {
    'C': [1,10,20],
    'kernel': ['rbf','linear']
}, cv=5, return_train_score=False)
clf.fit(iris.data, iris.target)
clf.cv_results_

**Step7.1: Converting results into dataframe**

In [ ]:
df = pd.DataFrame(clf.cv_results_)
df

**Step7.2: Viewing only the required coloumns**

In [ ]:
df[['param_C','param_kernel','mean_test_score']]

**Step7.3: Viewing best parameters**

In [ ]:
clf.best_params_

**Step7.4: Viewing best score**

In [ ]:
clf.best_score_

**Step8: Use RandomizedSearchCV to reduce number of iterations and with random combination of parameters. This is useful when you have too many parameters to try and your training time is longer. It helps reduce the cost of computation**

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(svm.SVC(gamma='auto'), {
        'C': [1,10,20],
        'kernel': ['rbf','linear']
    },
    cv=5,
    return_train_score=False,
    n_iter=2
)
rs.fit(iris.data, iris.target)
pd.DataFrame(rs.cv_results_)[['param_C','param_kernel','mean_test_score']]

**Step9: How about different models with different hyperparameters?**

In [ ]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
# Iitialising model_params dictionary.Each model is defined as a key-value pair in the dictionary.
# The key represents the models name, and the value is another dictionary that contains the model
# object and its corresponding hyperparameter options.
model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    }
}


**Step9.1: Applying loop on the dictionary**

In [ ]:
scores = []
# model_name is the key and mp is vlaue of the above dictionary
# Example svm is model name and mp is the dictionary value of svm
for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(iris.data, iris.target)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })

df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

**Based on above, I can conclude that SVM with C=1 and kernel='rbf' is the best model for solving my problem of iris flower classification**